<img src="imgs/IISc_Master_Seal_Black_Transparent.png" height="120px" width="120px" align="right" />

<img src="imgs/logoGESS.jpg" height="120px" width="120px" align="right" />

<font face="Calibri">
<font size="7"> <b> PySHBundle Tutorials </b> </font>
<br> 
<font size="5"> <b> Tutorial 3: Terrestrial Water Storage Time Series<font color='rgba(200,0,0,0.2)'>  </font> </b> </font>
<br> <br>
    
<font size="3"> <b> by: Abhishek Mhamane, IIT Kanpur</b> 
<font size="2">  <br>
<font> <b>Date: </b> August 30, 2023 </font>
</font>


In [15]:
import os
import pprint
import numpy as np
from tqdm import tqdm, trange
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
os.chdir('D:\\open_source\\pyshbundle')
import pyshbundle

In [11]:
from pyshbundle.new_io import read_jpl, read_csr, read_tn13, read_tn14, extract_C10_11_replcmnt_coeff, klm2sc_new
from pyshbundle.new_io import extract_C20_replcmnt_coeff, extract_C30_replcmnt_coeff, replace_zonal_coeff, find_date_in_replacemnt_file
from pyshbundle.new_io import read_itsg

from pyshbundle import gshs, plm, sc2cs
from pyshbundle.clm2sc import clm2sc_new
from pyshbundle.clm2cs import clm2cs_new

In [ ]:
# load temporal data

In [4]:
def last_4chars(x):
        #print(x[-39:-32])
        return(x[-11:-4])

In [5]:
tn14_path_itsg = 'D:\\open_source\\pyshbundle\pyshbundle\\data\\ITSG_TN_files\\TN-14_C30_C20_SLR_GSFC.txt'
tn_13_path_itsg = 'D:\\open_source\\pyshbundle\\pyshbundle\\data\\ITSG_TN_files\\TN-13_GEOC_CSR_RL06.1.txt'
itsg_folder = 'D:\\open_source\\pyshbundle\\sample_input_data\\ITSG_input'
itsg_file = 'D:\\open_source\\pyshbundle\\sample_input_data\\ITSG_input\\ITSG-Grace2018_n96_2011-05.gfc'

In [6]:
# load all the itsg files
files = os.listdir(itsg_folder)
files = sorted(files, key = last_4chars)

data_itsg_mat = np.ndarray((len(files), 4753, 6))
dates_itsg = []
for i in trange(len(files)):
    file_path = itsg_folder + '\\' + files[i]
    header = read_itsg(file_path)[0]
    data_itsg_mat[i, :, :] = read_itsg(file_path)[1]
    dates_itsg.append(read_itsg(file_path)[2])

100%|██████████| 214/214 [00:24<00:00,  8.63it/s]


In [8]:
# Read replacemnt file
data_tn13_itsg = read_tn13(tn_13_path_itsg)
data_tn14_itsg = read_tn14(tn14_path_itsg)

In [ ]:
header

In [ ]:
type(read_itsg(file_path)[2])

In [9]:
#replace_zonal_coeff(data_mat, data_tn13, data_tn14, epoch_begin, epoch_end)
# Replace zonal coeff
replaced_data_mat_itsg = np.ndarray((len(files), 4753, 6))

unreplaced_idx = set()
for i in trange(len(files)):
    try:
        replaced_data_mat_itsg[i, :, :] = replace_zonal_coeff(data_itsg_mat[i, :, :], 'itsg', 96, data_tn13_itsg, data_tn14_itsg, dates_itsg[i], epoch_end=None)
        print("\n")
    except IndexError:
        print(f"Replacement data for File Index {i} - {dates_itsg[i]} could not be found.... procedding to next file")
        unreplaced_idx.add(i)

unreplaced_idx = list(unreplaced_idx)

  3%|▎         | 6/214 [00:00<00:04, 51.78it/s]

Data Date - 2002-04-01, replacemnt date(tn-13) = 2002-04-05
Data Date - 2002-04-01, replacemnt date(tn-13) = 2002-04-05
[1.00000000e+00 0.00000000e+00 5.21793863e-10 0.00000000e+00
 4.45850000e-11 0.00000000e+00] [ 1.00000000e+00  1.00000000e+00  2.86777277e-10 -6.80704629e-11
  4.52960000e-11  5.07240000e-11]
Data Date - 2002-04-01, replacemnt date (tn-14)_ = 2002-04-04
Data Date - 2002-04-01, replacemnt date (tn-14)_ = 2002-04-04


Data Date - 2002-05-01, replacemnt date(tn-13) = 2002-05-03
Data Date - 2002-05-01, replacemnt date(tn-13) = 2002-05-03
[1.00000000e+00 0.00000000e+00 4.69850864e-10 0.00000000e+00
 4.45850000e-11 0.00000000e+00] [ 1.00000000e+00  1.00000000e+00  1.53669799e-10 -8.90896621e-11
  4.52960000e-11  5.07240000e-11]
Data Date - 2002-05-01, replacemnt date (tn-14)_ = 2002-05-02
Data Date - 2002-05-01, replacemnt date (tn-14)_ = 2002-05-02


Data Date - 2002-08-01, replacemnt date(tn-13) = 2002-08-01
Data Date - 2002-08-01, replacemnt date(tn-13) = 2002-08-01
[1.0

  9%|▉         | 19/214 [00:00<00:03, 57.09it/s]



Data Date - 2003-07-01, replacemnt date(tn-13) = 2003-07-01
Data Date - 2003-07-01, replacemnt date(tn-13) = 2003-07-01
[1.00000000e+00 0.00000000e+00 9.70078027e-12 0.00000000e+00
 4.45850000e-11 0.00000000e+00] [ 1.00000000e+00  1.00000000e+00  4.57600314e-11 -3.88527543e-13
  4.52960000e-11  5.07240000e-11]
Data Date - 2003-07-01, replacemnt date (tn-14)_ = 2003-07-01
Data Date - 2003-07-01, replacemnt date (tn-14)_ = 2003-07-01


Data Date - 2003-08-01, replacemnt date(tn-13) = 2003-08-01
Data Date - 2003-08-01, replacemnt date(tn-13) = 2003-08-01
[ 1.00000000e+00  0.00000000e+00 -8.48334144e-11  0.00000000e+00
  4.45850000e-11  0.00000000e+00] [1.00000000e+00 1.00000000e+00 8.99940877e-12 1.56049408e-10
 4.52960000e-11 5.07240000e-11]
Data Date - 2003-08-01, replacemnt date (tn-14)_ = 2003-08-01
Data Date - 2003-08-01, replacemnt date (tn-14)_ = 2003-08-01


Data Date - 2003-09-01, replacemnt date(tn-13) = 2003-09-01
Data Date - 2003-09-01, replacemnt date(tn-13) = 2003-09-01
[ 

 14%|█▍        | 31/214 [00:00<00:04, 43.40it/s]

...



Data Date - 2019-10-01, replacemnt date(tn-13) = 2019-10-01
Data Date - 2019-10-01, replacemnt date(tn-13) = 2019-10-01
[ 1.00000000e+00  0.00000000e+00 -7.59338771e-10  0.00000000e+00
  4.45850000e-11  0.00000000e+00] [ 1.00000000e+00  1.00000000e+00 -2.42598479e-10  2.12460174e-10
  4.52960000e-11  5.07240000e-11]
Data Date - 2019-10-01, replacemnt date (tn-14)_ = 2019-10-01
Data Date - 2019-10-01, replacemnt date (tn-14)_ = 2019-10-01


Data Date - 2019-11-01, replacemnt date(tn-13) = 2019-11-01
Data Date - 2019-11-01, replacemnt date(tn-13) = 2019-11-01
[ 1.00000000e+00  0.00000000e+00 -6.08903465e-10  0.00000000e+00
  4.45850000e-11  0.00000000e+00] [ 1.00000000e+00  1.00000000e+00 -2.40374098e-10  1.74746338e-10
  4.52960000e-11  5.07240000e-11]
Data Date - 2019-11-01, replacemnt date (tn-14)_ = 2019-11-01
Data Date - 2019-11-01, replacemnt date (tn-14)_ = 2019-11-01


Data Date - 2019-12-01, replacemnt date(tn-13) = 2019-12-01
Data Date - 2019-12-01, replacemnt date(tn-13) = 2

 93%|█████████▎| 198/214 [00:03<00:00, 60.69it/s]



Data Date - 2020-11-01, replacemnt date(tn-13) = 2020-11-01
Data Date - 2020-11-01, replacemnt date(tn-13) = 2020-11-01
[ 1.00000000e+00  0.00000000e+00 -5.97818701e-10  0.00000000e+00
  4.45850000e-11  0.00000000e+00] [ 1.00000000e+00  1.00000000e+00 -2.19201402e-10  3.31000968e-10
  4.52960000e-11  5.07240000e-11]
Data Date - 2020-11-01, replacemnt date (tn-14)_ = 2020-11-01
Data Date - 2020-11-01, replacemnt date (tn-14)_ = 2020-11-01


Data Date - 2020-12-01, replacemnt date(tn-13) = 2020-12-01
Data Date - 2020-12-01, replacemnt date(tn-13) = 2020-12-01
[ 1.00000000e+00  0.00000000e+00 -5.68793295e-10  0.00000000e+00
  4.45850000e-11  0.00000000e+00] [ 1.00000000e+00  1.00000000e+00 -1.80358603e-10  2.38154881e-10
  4.52960000e-11  5.07240000e-11]
Data Date - 2020-12-01, replacemnt date (tn-14)_ = 2020-12-01
Data Date - 2020-12-01, replacemnt date (tn-14)_ = 2020-12-01


Data Date - 2021-01-01, replacemnt date(tn-13) = 2021-01-01
Data Date - 2021-01-01, replacemnt date(tn-13) = 2

100%|██████████| 214/214 [00:03<00:00, 57.71it/s]



Data Date - 2021-11-01, replacemnt date(tn-13) = 2021-11-01
Data Date - 2021-11-01, replacemnt date(tn-13) = 2021-11-01
[ 1.00000000e+00  0.00000000e+00 -7.73912212e-10  0.00000000e+00
  4.45850000e-11  0.00000000e+00] [ 1.00000000e+00  1.00000000e+00 -2.97545947e-10  2.67740152e-10
  4.52960000e-11  5.07240000e-11]
Data Date - 2021-11-01, replacemnt date (tn-14)_ = 2021-11-01
Data Date - 2021-11-01, replacemnt date (tn-14)_ = 2021-11-01


Data Date - 2021-12-01, replacemnt date(tn-13) = 2021-12-01
Data Date - 2021-12-01, replacemnt date(tn-13) = 2021-12-01
[ 1.00000000e+00  0.00000000e+00 -6.77281949e-10  0.00000000e+00
  4.45850000e-11  0.00000000e+00] [ 1.00000000e+00  1.00000000e+00 -2.34335947e-10  1.84325896e-10
  4.52960000e-11  5.07240000e-11]
Data Date - 2021-12-01, replacemnt date (tn-14)_ = 2021-12-01
Data Date - 2021-12-01, replacemnt date (tn-14)_ = 2021-12-01


Data Date - 2022-01-01, replacemnt date(tn-13) = 2022-01-01
Data Date - 2022-01-01, replacemnt date(tn-13) = 2

In [18]:
scmat_deck_itsg = np.ndarray((len(files), 97, 193))

for i in trange(196):
    if i not in unreplaced_idx:
        scmat_deck_itsg[i, :, :] = clm2sc_new(replaced_data_mat_itsg[i, :, :], lmax=96)
    
    

100%|██████████| 196/196 [00:01<00:00, 175.04it/s]


In [25]:
scmat_deck_itsg = np.ndarray((len(files), 97, 193))

for i in trange(196):
    if i not in unreplaced_idx:
        scmat_deck_itsg[i, :, :] = clm2sc_new(data_itsg_mat[i], lmax=96)
    
    

100%|██████████| 196/196 [00:00<00:00, 198.44it/s]


## ITSG Mean for the time period

In [21]:
temporal_mean_itsg = 0
for i in trange(196):
    temporal_mean_itsg += scmat_deck_itsg[i, :, :]

temporal_mean_itsg /= 196


100%|██████████| 196/196 [00:00<00:00, 59232.19it/s]


### Substract the mean from temopral data

In [22]:
sub_mat_itsg = np.ndarray((196, 97, 193))
for i in range(196):
    sub_mat_itsg[i, :, :] = scmat_deck_itsg[i, :, :] - temporal_mean_itsg

# Functions

In [23]:
from pyshbundle import gshs
def gshs_prepare(lmax, gs, quant, grd, h, jflag, sc_coeff):
    n = int(180/gs)
    
    grid_y = int(180/gs)
    grid_x = int(360/gs)

    ff = gshs.gshs(sc_coeff, quant, grd, n, h, jflag)[0]

    # rearranging
    field = np.zeros([grid_y,grid_x], dtype ='longdouble')

    field[:,0:int(grid_x/2)] = ff[:,int(grid_x/2):]
    field[:,int(grid_x/2):] = ff[:,0:int(grid_x/2)]  
    
    return field

In [24]:
def apply_gaussian(sc_coeff, gaussian_coeff, lmax):
    
    # filtered SH Coeff
    shfil = np.zeros([lmax+1, 2 * lmax+1])

    # applying filter on substracted coeff
    for j in range(0, 2*lmax+1, 1):
        shfil[:,j] = gaussian_coeff[:,0] * sc_coeff[:,j]
    
    return shfil

### Function trial

In [ ]:
scmat_deck_itsg = np.ndarray((len(files), 97, 193))

for i in trange(196):
    if i not in unreplaced_idx:
        scmat_deck_itsg[i, :, :] = clm2sc_new(data_itsg_mat[i], lmax=96)
    
    

100%|██████████| 196/196 [00:00<00:00, 211.15it/s]
